In [ ]:
!git clone https://github.com/arolihas/ppi-motifs.git
!git clone https://github.com/benedekrozemberczki/OrbitalFeatures.git

fatal: destination path 'ppi-motifs' already exists and is not an empty directory.
fatal: destination path 'OrbitalFeatures' already exists and is not an empty directory.


In [ ]:
!pip install texttable
!pip install python-igraph
!pip install cairocffi

In [ ]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import pandas as pd
from texttable import Texttable
from tqdm import tqdm
from networkx.generators.atlas import *
import igraph as ig
from igraph import Graph
from collections import defaultdict
import cairocffi

In [ ]:
G = Graph.Read_GML('/content/ppi-motifs/networks/full-ppi')

In [ ]:
class MotifCounter():

    def __init__(self, graph, size, cut_prob):
        self.graph = graph
        self.size = size
        self.motifDict = defaultdict(list)
        self.cut_prob = cut_prob

    def counter(self, graph, vertices, motif):
        self.motifDict[motif].append(vertices)

    def count(self):
        Graph.motifs_randesu(self.graph, size=self.size, cut_prob=self.cut_prob, callback = self.counter)

    def enumerate_categories(self):
        main_index = 0
        self.orbital_position = dict()
        for index,key in enumerate(self.motifDict.keys()):
            self.orbital_position[key] = dict()
            degrees = set(Graph.Isoclass(self.size, key).degree())
            for degree in degrees:
                self.orbital_position[key][degree] = main_index
                main_index = main_index + 1
        self.unique_motif_count = main_index

    def count_orbital_position(self):
        self.features = {node['label']: {i:0 for i in range(self.unique_motif_count)} for node in self.graph.vs}
        for size, node_lists in self.motifDict.items():
            for nodes in tqdm(node_lists):
                subgraph = self.graph.induced_subgraph(nodes, "create_from_scratch")
                subgraph = subgraph.simplify()
                degrees = subgraph.degree()
                for idx, v in enumerate(subgraph.vs):
                    degree = degrees[idx]
                    position = self.orbital_position[size][degree]
                    self.features[v['label']][position] += 1
                    
    def create_tabular_motifs(self):
         self.motifs = [[n['label']]+[self.features[n['label']][i] for i in  range(self.unique_motif_count)] for n in self.graph.vs()]
         self.motifs = pd.DataFrame(self.motifs)
         self.motifs.columns = ["id"] + ["role_"+str(index) for index in range(self.unique_motif_count)]

In [ ]:
model = MotifCounter(G, 3, [0.9,0.8,0.7])
model.count()
model.enumerate_categories()

In [ ]:
model.orbital_position

{2: {1: 1, 2: 2}, 3: {2: 0}}

In [ ]:
model.count_orbital_position()

100%|██████████| 292766/292766 [00:08<00:00, 34699.76it/s]


In [ ]:
model.create_tabular_motifs()

In [ ]:
model.motifs

,id,role_0,role_1,role_2
0,1394,0,0,0
1,2778,0,0,0
2,6331,0,0,0
3,17999,0,0,0
4,122704,0,0,0
...,...,...,...,...
21552,19400,0,0,0
21553,81918,0,0,0
21554,139378,0,1,0
21555,18744,0,2,0
